# Aplicación de prueba

In [1]:
import pandas as pd
import joblib

from datetime import datetime
from flask import Flask, request


In [2]:
# Definiciones globales y funciones de transformación

%run ../Lib/pandas_options.py
%run ../Lib/define_constantes.py
%run ../Lib/fn_transformaciones.py


Fecha y Hora de ejecución de módulo: 2024-09-26 09:30:27


In [3]:
# Cargar archivo de tramos

archivo_tramos = f'{DIRECTORIO_DATOS}/DATOSTRAMOS_2023-08-06.csv'
df_tramos = pd.read_csv(archivo_tramos, low_memory=False)

print(df_tramos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205754 entries, 0 to 205753
Data columns (total 48 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   DIAMETRO    205754 non-null  int64  
 1   CAMARA_CAI  205754 non-null  object 
 2   CONEXIONES  97049 non-null   object 
 3   MATERIAL    205377 non-null  object 
 4   TUBERIA_EL  197356 non-null  object 
 5   VIDA_UTIL   195982 non-null  float64
 6   FABRICANTE  190888 non-null  object 
 7   TIPO_CIMEN  83807 non-null   object 
 8   TIPO_SUELO  65523 non-null   object 
 9   TIPO_RED    205754 non-null  object 
 10  PROF_BATEA  204504 non-null  float64
 11  PROF_BATE1  204369 non-null  float64
 12  COTA_BATEA  202933 non-null  float64
 13  COTA_BATE1  202919 non-null  float64
 14  IPID        205754 non-null  int64  
 15  ESTADO      205754 non-null  object 
 16  GRUPO       205754 non-null  object 
 17  PROPIETARI  205734 non-null  object 
 18  OPERADOR    205512 non-null  object 
 19  EM

In [4]:
# Cargar modelo

archivo_modelo = f'{DIRECTORIO_MODELOS}/modelo.joblib'
modelo = joblib.load(archivo_modelo)

print(modelo)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='gpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=0.05, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)


In [5]:
# Cargar columnas

columnas_modelo = []
archivo_columnas = f'{DIRECTORIO_MODELOS}/COLUMNAS.txt'

with open(archivo_columnas, "r") as archivo:
    for linea in archivo:
        cadena = linea.strip()
        columnas_modelo.append(cadena)

print(columnas_modelo)


['EDAD', 'DIAMETRO', 'LONGITUD', 'AREA', 'PROF_BATEA', 'PROF_BATE1', 'PENDIENTE', 'ARRANQUE', 'ZONA_SUR', 'MUNICIPIO_MEDELLIN', 'ESTADO_OPERACION', 'TIPO_AGUA_COMBINADAS', 'TIPO_AGUA_LLUVIAS Y DESCARGAS', 'TIPO_AGUA_OTRAS', 'MATERIAL_CONCRETO REFORZADO Y OTROS', 'MATERIAL_CONCRETO SIMPLE', 'MATERIAL_PVC', 'FABRICANTE_INDUSTRIAS DIQUE S.A.', 'FABRICANTE_MEXICHEM COLOMBIA S.A.S.', 'FABRICANTE_OTROS']


In [6]:
# Generar predicción

def generar_prediccion(ipid):

    df_predecir = df_tramos[df_tramos['IPID'] == ipid].copy()

    # Agregar columnas dummy para codificacion
    df_predecir['CALIFICACION'] = 1 # Irrelevante, no se usa pero es requerido para transformacion
    df_predecir['FECHA_CALIFICACION'] = datetime.now()
  
    X_predecir = tx_codificar(tx_aplicar_transformaciones(df_predecir))

    # Agregar columnas faltantes
    for col in columnas_modelo:
        if col not in X_predecir.columns:
            print(f'Agregando {col}')
            X_predecir[col] = False
    
    # Reordenar columnas
    X_predecir = X_predecir[columnas_modelo]
    
    probabilidades = modelo.predict_proba(X_predecir)
    probabilidad_deterioro = probabilidades[:, 1]
        
    return f'Probabilidad de estado DETERIORADO en tramo {ipid}: {probabilidad_deterioro[0]:.2%}'
    

In [7]:
# Generar mensaje de respuesta

def generar_respuesta(texto_consulta):
    try:
        ipid = int(texto_consulta)
    except ValueError:
        return 'IPID debe ser numérico'
        
    if ipid in df_tramos['IPID'].values:
        return generar_prediccion(ipid)
    else:
        return f"No encuentro tramo {ipid} en archivo de tramos"
    

In [8]:
# Prototipo de servicio de consulta

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def predecir():
    if request.method == 'POST':
        texto_consulta = request.form['ipid']
        mensaje = generar_respuesta(texto_consulta)
        return f'''
        <p>{mensaje}</p>
        <form method="GET">
            <input type="submit" value="Consultar otro IPID">
        </form>
        '''
    else:
        return '''
        <h1>PREDICCIÓN DE ESTADO DE TRAMOS DE TUBERÍA</h1>
        <form method="POST">
            <label for="ipid">Ingresa el IPID del tramo:</label>
            <input type="text" id="ipid" name="ipid" required>
            <input type="submit" value="Enviar">
        </form>
        '''

if __name__ == '__main__':
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Sep/2024 09:32:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Sep/2024 09:32:48] "GET /favicon.ico HTTP/1.1" 404 -
